<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [157]:
import pandas as pd
import psycopg2

In [159]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [160]:
# текст запроса
query_3_1 = f''' select count(*) from vacancies
'''

In [161]:
# результат запроса
pd.read_sql_query(query_3_1, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/1469115235.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [162]:
# текст запроса
query_3_2 = f''' select count(distinct id) from employers
'''



In [163]:
# результат запроса
pd.read_sql_query(query_3_2,connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/2966244997.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [164]:
# текст запроса
query_3_3 = f''' select count(distinct id) from areas
'''


In [165]:
# результат запроса
pd.read_sql_query(query_3_3,connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/3873399239.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [166]:
# текст запроса
query_3_4 = f''' select count(distinct id) from industries
'''


In [167]:
# результат запроса
pd.read_sql_query(query_3_4,connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/1330702017.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,294


***

# выводы по предварительному анализу данных
по предварительному анализу можно сделать вывод что всего на тот момент на рынке труда было около 49 тысяч вакансий по 290 различным направлениям. из 49 тысяч вакансий имеется 29 тысяч работодателей, логично предположить, что большинство работодателей предлагают несколько вакансий, в детальном анализе это будет доказано.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [168]:
# текст запроса 
query_4_1 = f''' select a.name, count(*)
from vacancies v left join areas a on v.area_id = a.id 
group by a.name
order by 2 desc
'''

In [169]:
# результат запроса
pd.read_sql_query(query_4_1, connection)    

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/3847003360.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [170]:
# текст запроса
query_4_2 = ''' select count(*)
from vacancies where salary_from is not null or salary_to is not null


'''

In [171]:
# результат запроса
pd.read_sql_query(query_4_2, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/614600247.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [172]:
# текст запроса

# Конечно через пандас вычислить эти средние значения можно было бы в разы быстрее и легче, но проект то про SQL :)
query_4_3 = ''' with x as (select salary_from salary, 'минималка' border_type
from vacancies where salary_from is not null
union all
select salary_to, 'максималка'
from vacancies where salary_to is not null)

select 'Минималка среднее:' mean_type , Round(Sum(salary)/count(*)::float) mean_value 
from x where border_type = 'минималка'
union 
select 'Максималка среднее:', Round(Sum(salary)/count(*)::float) 
from x where border_type = 'максималка'
'''

In [173]:
# результат запроса
pd.read_sql_query(query_4_3, connection)


/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/3354881684.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,mean_type,mean_value
0,Максималка среднее:,110537.0
1,Минималка среднее:,71065.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [174]:
# текст запроса 
query_4_4 = '''select schedule, employment, count(*) cnt
from vacancies
group by schedule, employment
order by count(*) desc
'''

In [175]:
# результат запроса
pd.read_sql_query(query_4_4, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/2630570843.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [176]:
# текст запроса 
query_4_5 = '''select experience, count(*) cnt
from vacancies 
group by experience
order by count(*) asc
'''

In [177]:
# результат запроса
pd.read_sql_query(query_4_5, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/3190483664.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [178]:
# вывод графика для детального анализа
import plotly.express as px
px.bar(pd.read_sql_query(query_4_5, connection), x='experience', y='cnt', title='График количества вакансий по опыту работы')


/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/1009382269.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



***

# Вывод по детальному анализу 
По данному графику можно сделать вывод, что наибольшее количество вакансий
предполагает опыт работы от 1 до 3 лет, а также от 3 до 6 лет.
То есть, большинство вакансий ищут **Junior+/Middle** и **Strong Middle** специалистов.

По первому запросу же, можно сделать вывод, что наибольшее количество вакансий ищут работников в Москве
А так же наиболее популярный тип графика и занятости - полный рабочий день и постоянная занятость

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [179]:
# текст запроса

query_5_1 = '''with x as (select e.name, count(*) cnt
from vacancies v left join employers e on v.employer_id = e.id
group by e.name
order by 2 desc)

(select name, cnt from x limit 1)
union (select name, cnt from x offset 4 limit 1)
'''
# думаю разом запросить таблицу и потом оттуда просто выбрать 1 и 5 элементы будет быстрее, чем делать 2 запроса 

In [180]:
# результат запроса

pd.read_sql_query(query_5_1, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/1268079039.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt
0,Газпром нефть,331
1,Яндекс,1933


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [181]:
# текст запроса

#Запрос получился достаточно тяжелым, у меня он выполнился за 20 секунд, возможно что-то можно было сделать проще, но оно работает)
test_query = ''' 
    with x as (SELECT 
        a.name AS area_name,
        COUNT(DISTINCT e.id) AS employer_cnt,
        COUNT(v.id) AS vacancy_cnt
    FROM areas a
    LEFT JOIN employers e ON a.id = e.area
    LEFT JOIN vacancies v ON a.id = v.area_id
    GROUP BY a.id, a.name)
    
    select area_name, employer_cnt from x where vacancy_cnt = 0
    order by employer_cnt desc
'''

In [182]:
# результат запроса
pd.read_sql_query(test_query, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/3143202613.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area_name,employer_cnt
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Ростовская область,18
...,...,...
588,Скидель,0
589,Белоозерск,0
590,Орловский,0
591,Павловская,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [183]:
# текст запроса
query_5_3 = '''
select e.name, count(DISTINCT v.area_id)

from employers e left join vacancies v on e.id = v.employer_id
join areas a on a.id = e.area
group by e.name
order by 2 desc
'''

In [184]:
# результат запроса
pd.read_sql_query(query_5_3, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/593965358.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [185]:
# текст запроса
query_5_4 = '''
     SELECT COUNT(e.id) AS employers_without_industry
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL;

'''

In [186]:
# результат запроса
pd.read_sql_query(query_5_4, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/1644714913.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employers_without_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [187]:
# текст запроса
query_5_5 = '''
SELECT e.name, count(*)
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
LEFT JOIN industries i on i.id = ei.industry_id
group by e.name
having count(*) = 4
order by 1 asc
offset 2 limit 1
'''

In [188]:
# результат запроса
pd.read_sql_query(query_5_5, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/1169902027.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [189]:
# текст запроса
query_5_6 = '''
select count(*)
from employers e left join employers_industries ei on ei.employer_id = e.id
left join industries i on i.id = ei.industry_id 
where i.name = 'Разработка программного обеспечения'


'''

In [190]:
# результат запроса
pd.read_sql_query(query_5_6, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/2878008418.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [191]:
# код для получения списка городов-милионников
import requests 
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
tables = page.find_all("table")[0]

data = []
for row in tables.find_all("tr")[1:]:
    cells = row.find_all("td")
    if cells:
        data.append([cell.text.strip() for cell in cells])
data = [x[1] for x in data]

In [192]:
# текст запроса

query_5_7 = f'''
WITH x AS (
    SELECT a.name AS name, COUNT(v.id) AS cnt
    FROM vacancies v  
    JOIN employers e ON v.employer_id = e.id
    JOIN areas a ON a.id = v.area_id
    WHERE e.name = 'Яндекс' AND a.name IN ({", ".join(map(repr, data))})
    GROUP BY a.name
)
SELECT name, cnt FROM x
UNION ALL
SELECT 'Total', SUM(cnt) FROM x
ORDER BY cnt;
'''

In [193]:
# результат запроса
pd.read_sql_query(query_5_7, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/1216511710.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

# выводы по анализу работодателей

Как и предпологалось ранее, самой популярной компании с наибольшим количеством вакансий оказался - Яндекс.
При этом Яндекс вдобавок предлагает свои вакансии в 181 регионе. 

И среди городов милионников наиболее популярной по килечеству вакансий оказалась Москва.



# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [194]:
# текст запроса
query_6_1 = '''
select name from vacancies v where lower(name) like '%data%' or lower(name) like '%данн%'
'''


In [195]:
# результат запроса
pd.read_sql_query(query_6_1, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/2603769340.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name
0,Аналитик хранилищ данных и BI-систем/Аналитик ...
1,Data-analysts (удаленно)
2,Стажер Data Engineer/Data Аналитик в команду Б...
3,Старший специалист хранилищ данных
4,Администратор баз данных и сетей
...,...
1766,Программист Delphi / разработчик баз данных
1767,Разработчик баз данных (Big Data)
1768,Аналитик данных
1769,Data аналитик /Аналитик BI (удаленно)


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [196]:
# текст запроса
query_6_2 = '''

select count(*) from vacancies v where (lower(name) 
like '%data scientist%' 
or lower(name) like '%data science%'
or lower(name) like '%исследователь данных%' 
or (lower(name) like '%ml%' and lower(name) not like '%html%')
or lower(name) like '%machine learning%'
or lower(name) like '%машинн%обучен%' ) 

and 

(lower(name) like '%junior%' or experience = 'Нет опыта' or employment = 'Стажировка')
'''

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [197]:
# текст запроса
query_6_3 = '''
select count(*) from vacancies v where (lower(name) 
like '%data scientist%' 
or lower(name) like '%data science%'
or lower(name) = '%исследователь данных%' 
or (lower(name) like '%ml%' and lower(name) not like '%html%')
or lower(name) like '%machine learning%'
or lower(name) like '%машинн%обучен%' ) 

and 
(lower(key_skills) like '%sql%' or lower(key_skills) like '%postqres%')


'''

In [198]:
# результат запроса
pd.read_sql_query(query_6_3, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/3902879435.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [199]:
# текст запроса
query_6_4 = '''

select count(*) from vacancies v where (lower(name) 
like '%data scientist%' 
or lower(name) like '%data science%'
or lower(name) = '%исследователь данных%' 
or (lower(name) like '%ml%' and lower(name) not like '%html%')
or lower(name) like '%machine learning%'
or lower(name) like '%машинн%обучен%' ) 

and 
(lower(key_skills) like '%python%')


'''

In [200]:
# результат запроса
pd.read_sql_query(query_6_4, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/2548700122.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [201]:
# текст запроса
query_6_5 = '''
select round(avg(1 + length(key_skills) - length(replace(key_skills, CHR(9), ''))),2) as test 
from vacancies v 
where(v.name ILIKE '%data scientist%' OR 
v.name ILIKE '%data science%' OR
v.name ILIKE '%исследователь данных%' OR
(v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
v.name ILIKE '%machine learning%' OR
v.name ILIKE '%машинн%обучен%%') 


'''

In [202]:
# результат запроса
pd.read_sql_query(query_6_5, connection)


/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/972981902.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,test
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [203]:
# текст запроса
query_6_6 = '''
select v.experience, ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0)
from vacancies v
WHERE
(
       (v.name ILIKE '%data scientist%')
    OR (v.name ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name  LIKE '%ML%' AND  (v.name NOT iLIKE '%html%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)
GROUP BY v.experience
ORDER BY 1
'''

In [204]:
# результат запроса
pd.read_sql_query(query_6_6, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/1796641676.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,round
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,145236.0
3,От 3 до 6 лет,243115.0


***

# выводы по предметному анализу
По предметному анализу можно сделать вывод о том, что из 49 тысяч вакансий 530 вакансий связаным с DS.
При этом найдено 51 вакансии на позицию Junior DS. 

Так же стоит отметить, что среди 530 вакансий про DS 357 вакансий требует знание Python. На самом деле я считаю что цифра должна быть еще больше, ведь альтернатива для Python - R не особо популярен. Поэтому скорее всего или Python попросту не указали как обязательное требование, потому что предпологают, что Python должен знать любой DS и смысл писать Python как основные навыки нет.

Что касается зарплат: 
- **При опыте работы 1 - 3 года зарплата ожидается в районе 150 тясяч рублей.**
- **При опыте работы от 3 до лет зарплата будет насчитывать уже 240 тысяч рублей.**

Можно сделать вывод, что Junior разработчик минимально может получить около 150 тысяч рублей, а со временев став Junior+/Middle зарплата поднимется на 60% 



# Общий вывод по проекту

#### Общий вывод по проекту

В ходе данного проекта мы провели анализ данных о вакансиях и работодателях. Мы определили ключевые тенденции на рынке труда, выявили работодателей с наибольшим количеством вакансий и проанализировали распределение вакансий по различным параметрам. 

#### Основные выводы:
1. Работодатели с наибольшим количеством вакансий сосредоточены в крупных городах-милионниках и в определенных отраслях.
2. Большинство вакансий связанных с DS нацелены на поиск junior - middle специалистов
3. На основе данных по вакансиям можно сделать выводы о востребованности вакансий связанных с DS

#### Дополнительные исследования могут включать:
- Хотелось бы дополнительно проанализировать заработную плату не в зависимости от опыта работы, а от региона. Считаю это тоже очень важным показателем.
- Сравнить между собой средние ЗП вакансий связанных с DS и вакансий не связанных с DS


## Исследование зарплаты по городам

In [208]:

bonus_query = '''
select a.name as city_name, ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) as salary
from vacancies v 
left join areas a on a.id = v.area_id
group by a.name
having ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) is not null
order by 2 desc
limit 4
'''
px.pie(pd.read_sql_query(bonus_query, connection), values='salary', names='city_name', title='График средней зарплаты по городам')

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/4266057664.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



При анализе наибольших предлогаемых ЗП сгруппированных по городам в глаза сразу бросается тот факт, что лидеры предлогаемых заработных плат это сраны Европы. Если же чуть повысит топ стран до 7, то можно будет встретить уже и Россию :)

## Исследование зарплаты в области DS и зарплаты не в области DS

In [206]:
bunus_query_1 = '''
with y as (select v.name as vac_name, ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) as salary
from vacancies v 
WHERE
(
       (v.name not ILIKE '%data scientist%')
    OR (v.name not ILIKE '%data science%')
    OR (v.name not ILIKE '%исследователь данных%')
    OR (v.name  not LIKE '%ML%')
    OR (v.name not ILIKE '%machine learning%')
    OR (v.name not ILIKE '%машинн%обучен%')
)
group by v.name
having ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) is not null
order by 2 desc),

x as (select a.name as city_name, ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) as salary
from vacancies v 
left join areas a on a.id = v.area_id
WHERE
(
       (v.name ILIKE '%data scientist%')
    OR (v.name ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name  LIKE '%ML%' AND  (v.name NOT iLIKE '%html%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
)
group by a.name
having ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) is not null
order by 2 desc)

select 'Средние зарплаты у DS' as vac_name, avg(salary) from x
union 
select 'Средняя зарплата не у ДС', avg(salary) from y


'''
pd.read_sql_query(bunus_query_1, connection)

/var/folders/vc/zgch74_n23sckf5dm03_fq380000gn/T/ipykernel_1045/335458154.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vac_name,avg
0,Средние зарплаты у DS,153305.105263
1,Средняя зарплата не у ДС,89369.488880


# Вывод по данному запросу: 

При анализе зарплаты в разрезе сферы деятельности можно сделать вывод, что в целом среднестатистические вакансии IT проигрывают в больше чем в полтора раза средней зарплате вакансий связанной с DS. 
Да, если промежуточно взглянуть на анализ вакансий можно найти ряд вакансий которые сами по себе конкурирует со средней зарплатой DS, но так как большинство вакансий не про DS имеют меньшую зарплату, то и получая среднее значение расчетная зарплата вышла такой:

Вывод: 
- Вакансии связанные в DS в **большинстве** случаев прибыльнее других вакансий примерно в 1.75 раз!